In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,38332
2,Huelva,Confirmados PDIA 14 días,2852
3,Huelva,Tasa PDIA 14 días,"555,7612487090048"
4,Huelva,Confirmados PDIA 7 días,1376
5,Huelva,Tasa PDIA 7 dias,"268,13726445427443"
6,Huelva,Total Confirmados,38539
7,Huelva,Curados,33544
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  38332.0


/tmp/ipykernel_56540/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10691.0


/tmp/ipykernel_56540/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_56540/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_56540/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_56540/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 388 personas en los últimos 7 días 

Un positivo PCR cada 168 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,38332.0,1376.0,2852.0,513170.0,268.137264,555.761249,314.0
Huelva-Costa,22523.0,816.0,1807.0,289548.0,281.818559,624.076146,184.0
Condado-Campiña,12027.0,374.0,691.0,156231.0,239.389110,442.293783,101.0
Huelva (capital),10691.0,371.0,855.0,143837.0,257.930852,594.422854,66.0
Lepe,2516.0,90.0,185.0,27880.0,322.812052,663.558106,34.0
Sierra de Huelva-Andévalo Central,3408.0,185.0,353.0,67391.0,274.517369,523.808817,29.0
Isla Cristina,2612.0,82.0,183.0,21393.0,383.302950,855.419997,22.0
Ayamonte,1630.0,70.0,158.0,21104.0,331.690675,748.673237,22.0
Moguer,1634.0,68.0,125.0,21867.0,310.970869,571.637627,20.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,150.0,21.0,32.0,1310.0,1603.053435,2442.748092,0.0
Corteconcepción,34.0,9.0,10.0,536.0,1679.104478,1865.671642,2.0
Santa Ana la Real,19.0,3.0,8.0,475.0,631.578947,1684.210526,1.0
Cumbres Mayores,77.0,16.0,21.0,1749.0,914.808462,1200.686106,3.0
Beas,299.0,27.0,52.0,4341.0,621.976503,1197.880673,8.0
Punta Umbría,1049.0,68.0,137.0,15355.0,442.852491,892.217519,16.0
Isla Cristina,2612.0,82.0,183.0,21393.0,383.302950,855.419997,22.0
Palos de la Frontera,877.0,34.0,88.0,11742.0,289.558849,749.446432,8.0
Ayamonte,1630.0,70.0,158.0,21104.0,331.690675,748.673237,22.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Campillo (El),82.0,1.0,7.0,2024.0,49.407115,345.849802,0.0,0.142857
Zarza-Perrunal (La),41.0,2.0,7.0,1253.0,159.616919,558.659218,0.0,0.285714
Niebla,317.0,3.0,10.0,4116.0,72.886297,242.954325,1.0,0.300000
Bonares,358.0,9.0,28.0,6060.0,148.514851,462.046205,4.0,0.321429
Zalamea la Real,105.0,3.0,9.0,3054.0,98.231827,294.695481,0.0,0.333333
Escacena del Campo,164.0,3.0,9.0,2287.0,131.176213,393.528640,2.0,0.333333
Aroche,101.0,2.0,6.0,3054.0,65.487885,196.463654,0.0,0.333333
Valverde del Camino,753.0,21.0,61.0,12750.0,164.705882,478.431373,3.0,0.344262
San Bartolomé de la Torre,323.0,3.0,8.0,3761.0,79.766020,212.709386,2.0,0.375000
